In [15]:
import numpy as np
from gensim import models
import pandas as pd
from sklearn import metrics

# Load Word2Vec Model for getting word vectors

In [2]:
model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [3]:
print model.most_similar('books')

[(u'book', 0.7379178404808044), (u'tomes', 0.6916561126708984), (u'paperback_novels', 0.6453143358230591), (u'nonfiction_books', 0.6357849836349487), (u'novels', 0.6332962512969971), (u'booklist', 0.627798318862915), (u'textbooks', 0.6162418723106384), (u'ebooks', 0.6123335361480713), (u'Books', 0.6083732843399048), (u'hardbacks_paperbacks', 0.6080424189567566)]


# Load each subject and build the model

In [6]:
import scipy.io as sio
test = sio.loadmat('BrainDecoding/data-science-P9.mat')
print test['data'].shape

(360, 1)


In [7]:
words=[]
data = np.zeros([60,21764])

for i in range(0,360):
    words.append(test['info'][0][i][2][0])
len(words)
unique_words = set(words)
print len(unique_words)

60


In [8]:
print unique_words

set([u'bicycle', u'bell', u'house', u'church', u'telephone', u'ant', u'lettuce', u'carrot', u'beetle', u'table', u'skirt', u'bee', u'saw', u'airplane', u'barn', u'horse', u'apartment', u'eye', u'shirt', u'closet', u'cup', u'dress', u'arm', u'celery', u'window', u'chimney', u'knife', u'pants', u'butterfly', u'fly', u'door', u'chair', u'leg', u'corn', u'watch', u'screwdriver', u'bear', u'hand', u'glass', u'spoon', u'train', u'key', u'desk', u'foot', u'pliers', u'arch', u'refrigerator', u'tomato', u'cow', u'coat', u'chisel', u'dog', u'bed', u'cat', u'hammer', u'truck', u'bottle', u'car', u'dresser', u'igloo'])


# Generate word vectors for each word

In [10]:
X=[]
for i in unique_words:
    temp_list=[]
    word_vector = model.most_similar(i,topn=100)
    for j in range(0,100):
        temp_list.append(word_vector[j][1])
    X.append(temp_list)
X=np.array(X)
print X.shape

(60, 100)


In [11]:
print model.most_similar('celery',topn=100)

[(u'broccoli', 0.7052637934684753), (u'bell_peppers', 0.6974425315856934), (u'bell_pepper', 0.6973530054092407), (u'parsley', 0.6922789812088013), (u'cilantro', 0.6898031830787659), (u'cabbage', 0.6884720325469971), (u'cauliflower', 0.6757640838623047), (u'tomato', 0.6751956343650818), (u'lettuce', 0.6734380722045898), (u'cherry_tomatoes', 0.6692799925804138), (u'Brussels_sprouts', 0.663195013999939), (u'romaine_hearts', 0.6620377898216248), (u'basil', 0.6617282629013062), (u'chopped_celery', 0.6595742106437683), (u'curly_endive', 0.6584914922714233), (u'asparagus', 0.6569705009460449), (u'watercress', 0.6564053297042847), (u'rapini', 0.6559873223304749), (u'fresh_basil', 0.6558088064193726), (u'radicchio', 0.65521240234375), (u'cucumber', 0.6515625715255737), (u'fennel', 0.6470979452133179), (u'brussel_sprouts', 0.6466445326805115), (u'salad', 0.6456936597824097), (u'romaine', 0.64554762840271), (u'kale', 0.6455265283584595), (u'garlic', 0.6429183483123779), (u'chopped_kale', 0.642831

In [12]:
print test['data'][0][0][0].shape

(21344,)


In [15]:
sio.savemat('words',{'name':list(unique_words)})
sio.savemat('training_data',{'feat':y})

In [1]:
from sklearn.neural_network import MLPRegressor
reggg = MLPRegressor(activation='tanh')
print reggg

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


In [16]:
y=[]
k=0
for i in unique_words:
    temp_voxels =[]
    index=0
    for j in range(0,360):
        if i==test['info'][0][j][2][0] and index==0:
            y.append(test['data'][j][0][0])
            index=index+1
        elif i==test['info'][0][j][2][0]:
            y[k]=y[k]+test['data'][j][0][0]
            index=index+1
    y[k]=y[k]/6.0
    k=k+1
y=np.array(y)
print y.shape
avg = y - np.mean(y,axis=0)
print avg.shape

(60, 21344)
(60, 21344)


In [17]:
XX=[]
for i in X:
    XX.append(i)
YY=[]
for i in y:
    YY.append(i)
XX=np.array(XX)
YY=np.array(YY)
print XX.shape
print YY.shape

(60, 100)
(60, 21344)


# Word2Vec vectors to build the model

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import VotingClassifier
mse_word2vec=[]
for i in range(0,60):
    tempw1=list(XX[i+1:])
    tempw2 =list(XX[0:i])
    X_trainw = tempw2 + tempw1
    X_testw = np.array(XX[i])
    ytempw1=list(YY[i+1:])
    ytempw2 =list(YY[0:i])
    Y_trainw = ytempw2 + ytempw1
    Y_tew =np.array(YY[i])
    print Y_tew.shape
    reg = MLPRegressor(activation='tanh')
    reg.fit(X_trainw,Y_trainw)
    y_predw = reg.predict(X_testw)
    print np.array(y_predw[0]).shape
    mse_word2vec.append(r2_score(Y_tew, y_predw[0]))
    sio.savemat('subject9/word2vec/predicted'+str(i),{'vect':y_predf})
    sio.savemat('subject9/word2vec/actual'+str(i),{'vect':Y_tef})

In [22]:
fp = open('fasttext_vectors.txt','r')
lines = fp.readlines()
dict_data=[]
for eachline in lines:
    splitdata=eachline.split()
    temp = [float(i) for i in splitdata[1:101]]
    dict_data.append(temp)
print len(dict_data)
print np.array(dict_data).shape

60
(60, 100)


# FastText input vectors to build the model

In [249]:
mse_fasttext=[]
for i in range(0,60):
    tempf1=list(dict_data[i+1:])
    tempf2 =list(dict_data[0:i])
    X_trainf = tempf2 + tempf1
    X_testf = np.array(dict_data[i])
    ytempf1=list(YY[i+1:])
    ytemf2 =list(YY[0:i])
    Y_trainf = ytemf2 + ytempf1
    Y_tef = np.array(YY[i])
    print Y_tef.shape
    reg = MLPRegressor(activation='tanh')
    reg.fit(X_trainf,Y_trainf)
    y_predf = reg.predict(X_testf)
    mse_fasttext.append(r2_score(Y_tef, y_predf[0]))
    sio.savemat('subject9/fasttext/predicted'+str(i),{'vect':y_predf})
    sio.savemat('subject9/fasttext/actual'+str(i),{'vect':Y_tef})

(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


...

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


# Random vectors to build the model

In [250]:
random_data=np.random.rand(60,100)
mse_random=[]
for i in range(0,60):
    tempr1=list(random_data[i+1:])
    tempr2 =list(random_data[0:i])
    X_trainr = tempr2 + tempr1
    X_testr = np.array(random_data[i])
    ytempr1=list(YY[i+1:])
    ytemr2 =list(YY[0:i])
    Y_trainr = ytemr2 + ytempr1
    Y_ter = np.array(YY[i])
    print Y_ter.shape
    reg = MLPRegressor(activation='tanh')
    reg.fit(X_trainr,Y_trainr)
    y_predr = reg.predict(X_testr)
    mse_random.append(r2_score(Y_ter, y_predr[0]))
    sio.savemat('subject9/random/predicted'+str(i),{'vect':y_predr})
    sio.savemat('subject9/random/actual'+str(i),{'vect':Y_ter})

(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


...

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [19]:
fp1 = open('Mitchell_vectors.txt','r')
lines = fp1.readlines()
original_data=[]
for eachline in lines:
    splitdata=eachline.split()
    temp1 = [float(i) for i in splitdata[1:]]
    original_data.append(temp1)
    
print len(original_data)
original_data=np.array(original_data)
print original_data.shape

60
(60, 25)


In [20]:
X_new=[]
new_words = ['skull','heart','brain','parachute','tent','monkey','elephant']
for i in new_words:
    temp_list=[]
    word_vector = model.most_similar(i,topn=25)
    for j in range(0,25):
        temp_list.append(word_vector[j][1])
    X_new.append(temp_list)
X_new=np.array(X_new)
print X_new.shape

(7, 25)


# Mitchell's vectors to build the model

In [252]:
mse_org=[]
for i in range(0,60):
    tempo1=list(original_data[i+1:])
    tempo2 =list(original_data[0:i])
    X_traino = tempo2 + tempo1
    X_testo = np.array(original_data[i])
    ytempo1=list(YY[i+1:])
    ytemo2 =list(YY[0:i])
    Y_traino = ytemo2 + ytempo1
    Y_teo = np.array(YY[i])
    print Y_teo.shape
    reg = MLPRegressor(activation='tanh')
    reg.fit(X_traino,Y_traino)
    y_predo = reg.predict(X_testo)
    mse_org.append(r2_score(Y_teo, y_predo[0]))
    sio.savemat('subject9/original/predicted'+str(i),{'vect':y_predo})
    sio.savemat('subject9/original/actual'+str(i),{'vect':Y_teo})

(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


...

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(21344,)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [25]:
sio.savemat('subject9/mse_ori',{'vect':mse_org})
sio.savemat('subject9/mse_ens',{'vect':mse_ens})
sio.savemat('subject9/mse_fasttext',{'vect':mse_fasttext})
sio.savemat('subject9/mse_word2vec',{'vect':mse_word2vec})